In [10]:
import numpy as np
import tensorflow as tf
from skimage.feature import hog
from skimage import color
import os
import torch.utils.data as data
import torch

<b>Load complete CIFAR-10

In [11]:
# CIFAR-10 Image Category Dataset
# The CIFAR-10 data ( https://www.cs.toronto.edu/~kriz/cifar.html ) contains 60,000 32x32 color images of 10 classes.
# It was collected by Alex Krizhevsky, Vinod Nair, and Geoffrey Hinton.
CF10_Labels=['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']
(CF10_X_train, CF10_y_train), (CF10_X_test, CF10_y_test) = tf.contrib.keras.datasets.cifar10.load_data()

In [12]:
print(CF10_X_train.shape)
print(CF10_y_train.shape)
print(CF10_X_test.shape)
print(CF10_y_test.shape)

(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)


<b> Extract label pairs data

In [13]:
# The paper noted 5 label pairs
label_pair = [('deer','truck'), ('deer','horse'), ('automobile', 'truck'), ('cat', 'dog'), ('dog', 'horse')]

In [14]:
# get the index of label
def get_label_index(pair):
    label_pair = []
    label_pair.append(CF10_Labels.index(pair[0]))
    label_pair.append(CF10_Labels.index(pair[1]))
    return label_pair

In [15]:
# compute hog of image
def compute_hog(inputImage):
    image = color.rgb2gray(inputImage)
    fd = hog(image, orientations=8, pixels_per_cell=(8, 8), visualise=False, cells_per_block=(1, 1), block_norm='L2-Hys')
    MinMaxScale = (fd-np.min(fd))/(np.max(fd)-np.min(fd))
    return MinMaxScale
# compute color hist of image
def compute_color_hist(inputImage):
    np_hist,edges = np.histogramdd(inputImage.reshape(-1,3),bins=(3,3,3),normed=False,range=[(0,255),(0,255),(0,255)])
    MinMaxScale = (np_hist - np.min(np_hist))/(np.max(np_hist)-np.min(np_hist))
    return MinMaxScale.flatten()

In [23]:
# extract data from the complete data-set
# pre_processing the raw data into 155*1 vector: 128 hog + 27 color_hist
def extract_data(pair):
    pair_index = get_label_index(pair)
    
    # extract
    mask_index_train = (CF10_y_train[:,0] == pair_index[0]) | (CF10_y_train[:,0] == pair_index[1])
    X_train_raw = CF10_X_train[mask_index_train]
    y_train = CF10_y_train[mask_index_train]
    
    mask_index_test = (CF10_y_test[:,0] == pair_index[0]) | (CF10_y_test[:,0] == pair_index[1])
    X_test_raw = CF10_X_test[mask_index_test]
    y_test = CF10_y_test[mask_index_test]
    
    # change the label into 0 and 1
    mask_index_train = (y_train == pair_index[0])
    y_train[mask_index_train] = 0
    y_train[~mask_index_train] = 1
    
    mask_index_test = (y_test == pair_index[0])
    y_test[mask_index_test] = 0
    y_test[~mask_index_test] = 1
    
    # pre-process the X data
    X_train = np.zeros((X_train_raw.shape[0],155))
    X_test = np.zeros((X_test_raw.shape[0],155))
    for image in range(X_train_raw.shape[0]):
        hog = compute_hog(X_train_raw[image,:,:,:])
        color_hist = compute_color_hist(X_train_raw[image,:,:,:])
        tmp = np.concatenate((hog, color_hist),axis=0).reshape(1,-1)
        X_train[image] = tmp[0,:]
        
    for image in range(X_test_raw.shape[0]):
        hog = compute_hog(X_test_raw[image,:,:,:])
        color_hist = compute_color_hist(X_test_raw[image,:,:,:])
        tmp = np.concatenate((hog, color_hist),axis=0).reshape(1,-1)
        X_train[image] = tmp[0,:]
        
    return X_train, y_train, X_test, y_test

<b> Save data

In [20]:
def save_data(pair, X_train, y_train, X_test, y_test):
    data_path = 'data'
    if not(os.path.exists(data_path)):
        os.makedirs(data_path)
    file2save = os.path.join('data','CIFAR10_pair_{}_{}.npz'.format(pair[0], pair[1]))
    if not(os.path.exists(file2save)):
        np.savez_compressed(file2save,X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test)

<b> call functions to generate the real data-set

In [26]:
for i in range(len(label_pair)):
    X_train, y_train, X_test, y_test = extract_data(label_pair[i])
    save_data(label_pair[i], X_train, y_train, X_test, y_test)

<b>load the real data-set

In [11]:
def load_data(pair):
    savedfile = os.path.join('data','CIFAR10_pair_{}_{}.npz'.format(pair[0], pair[1]))
    if os.path.exists(savedfile):
        npzfile = np.load(savedfile)
        X_train, y_train, X_test, y_test = npzfile['X_train'], npzfile['y_train'], npzfile['X_test'], npzfile['y_test']
        npzfile.close()
    else:
        print("There doesn't exist this file, please generate is firstly!!")
        
    return X_train, y_train, X_test, y_test

In [12]:
X_train, y_train, X_test, y_test = load_data(('deer','truck'))

In [13]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(np.unique(y_train))
print(np.unique(y_test))

(10000, 155)
(10000, 1)
(2000, 155)
(2000, 1)
[0 1]
[0 1]


<b>generate batch data loader by using pytorch(cause our algorithm is realized by pytorch)

In [29]:
class MyDataset(data.Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __getitem__(self, index): # return tensor
        img, target = self.images[index], self.labels[index]
        return img, target

    def __len__(self):
        return len(self.images)

In [30]:
train_data = MyDataset(X_train, y_train)

In [31]:
print(train_data.images.shape)
print(train_data.labels.shape)

(10000, 155)
(10000, 1)


In [32]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=50, shuffle=True)

In [33]:
for step, (batch_X, batch_y) in enumerate(train_loader):
    print('Step: ', step, 'X.shape: ', batch_X.dtype, 'Y.shape: ', batch_y.dtype)

AttributeError: 'torch.DoubleTensor' object has no attribute 'dtype'